In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

# Data Load and other util function

In [2]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
#     print(cols_list)
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)

    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

# Function to build predictor and get performance

## Without CFS

In [33]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

## With CFS

In [27]:
def run_self_CFS(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = LogisticRegression()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

# Getting List of projects

In [28]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

# Run the experiment

In [34]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
with open('results/Performance/product_1000_nb.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

+++++++++++++++++   org.alloytools.alloy  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.65      0.97      0.78       138
           1       0.96      0.57      0.71       168

    accuracy                           0.75       306
   macro avg       0.80      0.77      0.74       306
weighted avg       0.82      0.75      0.74       306

+++++++++++++++++   qpython  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       164

    accuracy                           1.00       164
   macro avg       1.00      1.00      1.00       164
weighted avg       1.00      1.00      1.00       164

+++++++++++++++++   friendlychat-android  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         7

    accuracy                           1.00        10
   macro avg   

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37

    accuracy                           1.00        37
   macro avg       1.00      1.00      1.00        37
weighted avg       1.00      1.00      1.00        37

+++++++++++++++++   openshift-quickstarts  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.14      0.50      0.22         2
           1       0.80      0.40      0.53        10

    accuracy                           0.42        12
   macro avg       0.47      0.45      0.38        12
weighted avg       0.69      0.42      0.48        12

+++++++++++++++++   http-request  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   NumberProgressBar  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   eclipse.jdt.ls  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00  

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        64

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64

+++++++++++++++++   MifareClassicTool  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      0.83      0.91        18
           1       0.40      1.00      0.57         2

    accuracy                           0.85        20
   macro avg       0.70      0.92      0.74        20
weighted avg       0.94      0.85      0.88        20

+++++++++++++++++   RxJava2-Android-Samples  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.81      0.89      0.85        44
           1       0.55      0.40      0.46        15

    accuracy                           0.76        59
   macro avg       0.68      0.64      0.65        59
weighted

              precision    recall  f1-score   support

           0       0.33      0.25      0.29         4
           1       0.57      0.67      0.62         6

    accuracy                           0.50        10
   macro avg       0.45      0.46      0.45        10
weighted avg       0.48      0.50      0.48        10

+++++++++++++++++   Shuffle  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.71      0.67      0.69        15
           1       0.38      0.43      0.40         7

    accuracy                           0.59        22
   macro avg       0.54      0.55      0.54        22
weighted avg       0.61      0.59      0.60        22

+++++++++++++++++   AutoEx  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         4
           1       0.00      0.00      0.00         2

    accuracy                           0.67         6
   macro avg       0.33      0.50  

              precision    recall  f1-score   support

           0       0.91      0.74      0.82        43
           1       0.35      0.67      0.46         9

    accuracy                           0.73        52
   macro avg       0.63      0.71      0.64        52
weighted avg       0.82      0.73      0.76        52

+++++++++++++++++   chips-input-layout  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.94      0.59      0.73        56
           1       0.12      0.60      0.19         5

    accuracy                           0.59        61
   macro avg       0.53      0.59      0.46        61
weighted avg       0.88      0.59      0.68        61

+++++++++++++++++   elasticsearch-analysis-mmseg  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.77      0.67      0.71        30
           1       0.00      0.00      0.00         6

    accuracy                           0.56        36
  

...

              precision    recall  f1-score   support

           0       0.69      0.91      0.79      1087
           1       0.43      0.15      0.23       511

    accuracy                           0.66      1598
   macro avg       0.56      0.53      0.51      1598
weighted avg       0.61      0.66      0.61      1598

+++++++++++++++++   javacpp  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.64      0.82      0.72       283
           1       0.59      0.36      0.45       206

    accuracy                           0.63       489
   macro avg       0.61      0.59      0.58       489
weighted avg       0.62      0.63      0.60       489

+++++++++++++++++   XChart  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.60      0.79      0.68       594
           1       0.57      0.34      0.43       479

    accuracy                           0.59      1073
   macro avg       0.59      0.57  

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      3002
           1       0.23      0.26      0.25       456

    accuracy                           0.79      3458
   macro avg       0.56      0.56      0.56      3458
weighted avg       0.80      0.79      0.79      3458

+++++++++++++++++   Jest  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.70      0.80      0.74      1056
           1       0.39      0.27      0.32       503

    accuracy                           0.63      1559
   macro avg       0.54      0.53      0.53      1559
weighted avg       0.60      0.63      0.61      1559

+++++++++++++++++   flume  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.80      0.89      0.84      2021
           1       0.27      0.16      0.20       525

    accuracy                           0.74      2546
   macro avg       0.54      0.53      

              precision    recall  f1-score   support

           0       0.70      0.81      0.75       778
           1       0.34      0.22      0.27       350

    accuracy                           0.63      1128
   macro avg       0.52      0.51      0.51      1128
weighted avg       0.59      0.63      0.60      1128

+++++++++++++++++   streamex  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.75      0.65      0.70       557
           1       0.49      0.60      0.54       303

    accuracy                           0.63       860
   macro avg       0.62      0.63      0.62       860
weighted avg       0.66      0.63      0.64       860

+++++++++++++++++   testcontainers-java  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.71      0.91      0.80       660
           1       0.44      0.16      0.24       289

    accuracy                           0.68       949
   macro avg       0.

              precision    recall  f1-score   support

           0       0.58      0.86      0.69       392
           1       0.72      0.37      0.49       384

    accuracy                           0.62       776
   macro avg       0.65      0.61      0.59       776
weighted avg       0.65      0.62      0.59       776

+++++++++++++++++   spring-cloud-aws  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.58      0.84      0.69       645
           1       0.47      0.19      0.27       477

    accuracy                           0.57      1122
   macro avg       0.53      0.52      0.48      1122
weighted avg       0.54      0.57      0.51      1122

+++++++++++++++++   rtmp-rtsp-stream-client-java  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.72      0.74      0.73       608
           1       0.59      0.57      0.58       405

    accuracy                           0.67      1013
   m

              precision    recall  f1-score   support

           0       0.55      0.84      0.66      1308
           1       0.54      0.22      0.32      1148

    accuracy                           0.55      2456
   macro avg       0.55      0.53      0.49      2456
weighted avg       0.55      0.55      0.50      2456

+++++++++++++++++   digdag  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.59      0.88      0.70      1292
           1       0.55      0.19      0.29       996

    accuracy                           0.58      2288
   macro avg       0.57      0.54      0.49      2288
weighted avg       0.57      0.58      0.52      2288

+++++++++++++++++   openrouteservice  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.53      0.62      0.57       905
           1       0.72      0.64      0.68      1405

    accuracy                           0.63      2310
   macro avg       0.63   

In [30]:
for key in final_result.keys():
    print(len(final_result[key].values()))
    print(key,np.nanmedian(list(final_result[key].values())))

970
precision 0.51
970
recall 0.59
970
pf 0.32
970
f1 0.54
970
g 0.6
970
auc 0.6227185141890479
970
pci_20 0.62
970
ifa 1.0
